In [165]:
from typing import List
    

# class Crystal_inputBASE:
#     """
#     The base class of Crystal_input class
#     """
#     def __init__(self):
#         # Initialize the object to empty values
#         self._blocklist = ['_title', 
#                            '_block_geom', 
#                            '_block_opt', 
#                            '_block_optbs',
#                            '_block_freq', 
#                            '_block_anharm', 
#                            '_block_cphf', 
#                            '_block_elastcon', 
#                            '_block_eos', 
#                            '_block_bs',
#                            '_block_scf', 
#                            '_block_dft',
#                            '_block_d3',
#                            '_block_gcp']

class BlockBASE():
    """
    The base class of 'block' objects
    """
    def __init__(self):
        self._block_bg = ''
        self._block_ed = ''
        self._block_data = ''
        self._block_dict = {}
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)

    @property
    def data(self):
        """
        Settings in all the attributes are summarized here.
        """
        self.update_block()
        print('%s%s%s' % (self._block_bg, self._block_data, self._block_ed))
        return
    
    @staticmethod
    def assign_keyword(key, shape, value=None):
        """
        Transform value into string formats.
        
        Args:
            key (str): CRYSTAL keyword
            shape (list[int]): 1D list. Shape of input text. Length: Number of 
                lines; Element: Number of values
            value (list | str): If value =
                * list, a 1D list of arguments  
                * None or a list begins with None, return to keyword only  
                * '' or a list begins with '', Clean everything

        Returns:
            text (str): CRYSTAL input
        """
        if type(value) != list and type(value) != tuple:
            value = [value,]

        # Keyword only : Value is None and and key is not ''
        if value[0] == None and key != '':
            return '{}\n'.format(key)

        # Clean everything : Empty key or value is ''
        if value[0] == '' or key == '':
            return ''

        # Wrong input: Number of args defined by shape != input.
        if sum(shape) != len(value): 
            raise ValueError("The number of input parameters '{}' does not meet requirements.".format(value))

        # Correct number of args and valid key
        text = '{}\n'.format(key)
        value_counter = 0
        for nvalue in shape:
            for v in value[value_counter:value_counter + nvalue]:
                text += '{} '.format(v)
            text += '\b\n'
            value_counter += nvalue

        return text
    
    @staticmethod
    def set_matrix(mx):
        """
        Set matrix-like data to get assign_keyword inputs. Used for supercell 
        expansion matrix and strain tensor.
        
        Args:
            mx (list | str): ndimen \* ndimen list, None or ''
            
        Returns:
            shape (list): ndimen\*1 1D list. All elements are ndimen. Or [].
            value (list): ndimen\**2\*1 1D list. Flattened matrix, None or ''.
        """
        import numpy as np

        if mx == '': # Clean data
            return [], ''
        elif mx == None: # Keyword only
            return [], None
        
        matrix = np.array(mx)
        if matrix.shape[0] != matrix.shape[1]:
            raise ValueError("Input matrix is not a square matrix.")
        
        shape = [matrix.shape[0] for i in range(matrix.shape[0])]
        value = matrix.reshape([1, -1]).tolist()[0]

        return shape, value
    
    @staticmethod
    def set_list(*args):
        """
        Set list-like data to get assign_keyword inputs. Used for lists with
        known dimensions. Such as atom coordinate list.
        
        Args:
            \*args : If \*args is
                * '': Clean data. Shape = [], value = ''.
                * None: Keyword only. Shape = [], value = None.
                * *int, list*: int for length of the list, list for list data
        Returns:
            shape (list): 1 + length 1D list or []
            args (list): Flattened list, [] or ''
        """
        if args[0] == '': # Clean data
            return [], ''
        elif args[0] == None: # Keyword only
            return [], None

        if len(args) != 2 or int(args[0]) != len(args[1]):
            return InputeError('Input format error. Arguments should be int + list')
        
        shape = [1,]
        value = [int(args[0]),]
        
        if type(args[1][0]) == list or type(args[1][0]) == tuple: # 2D list (multi-rows)
            for i in args[1]:
                shape += [len(i),]
                value += i
        else: # 1D list (single row)
            shape += [len(args[0]),]
            value += args[0]
        
        return shape, value
    
    def clean_block(self):
        """
        Clean all the keyword-related attributes (accessible attributes).
        
        .. Note::
            This method directly deletes all the attributes. Alternatively, by
            setting an attribute with '', the attribute is kept but its old
            values are erased.
        """
        self._block_bg = ''
        self._block_ed = ''
        self._block_data = ''
        for a in self._block_attr:
            try: 
                delattr(self, a)
            except AttributeError:
                continue
        return

    def update_block(self):
        """
        Update the '_block_data' attribute: Summarizing all the settings to 
        '_block_data' attribute for inspection and print
        """
        if self._block_bg != '' or self._block_ed != '':
            self._block_data = ''
            for attr in self._block_attr:
                if hasattr(self, attr):
                    if attr[0] == '_': # Keyword-like attributes
                        self._block_data += getattr(self, attr)
                    else: # Block-like attributes
                        obj = getattr(self, attr)
                        obj.update_block()
                        self._block_data += obj._block_bg + obj._block_data + obj._block_ed
                else:
                    continue
        return

    def analyze_text(self, text):
        """
        Analyze the input text and return to corresponding attributes
        """
        import warnings
            
        textline = text.strip().split('\n')
        value = ''
        attr = ''
        nline = 0
        while nline < len(textline):
            line = textline[nline]
            if line in self._block_key: # keyword line
                if len(attr) > 0: # Assign the previous keyword
                    setattr(self, attr, value)
                
                # update attribute
                attr = self._block_dict[line]
                value = line + '\n'
                if hasattr(self, attr):
                    warnings.warn(
                        "Attribute '{}' exists. The new entry will cover the old one".format(attr), 
                        stacklevel=2
                    )
            else: # value line
                value += line + '\n'

            nline +=1
        
        return

class Geom(BlockBASE):
    """
    Geometry block object
    """
    def __init__(self):
        self._block_bg = 'Generated by CRYSTALpytools\n' # Set title as bg label
        self._block_ed = 'ENDGEOM\n'
        self._block_data = ''
        self._block_dict = {
            'CRYSTAL'   : '_basegeom',
            'SLAB'      : '_basegeom',
            'POLYMER'   : '_basegeom',
            'HELIX'     : '_basegeom',
            'MOLECULE'  : '_basegeom',
            'EXTERNAL'  : '_basegeom',
            'DLVINPUT'  : '_basegeom',
            'SUPERCEL'  : '_sp_matrix',
            'SUPERCELL' : '_sp_matrix',
            'SUPERCON'  : '_sp_matrix',
            'SCELCONF'  : '_sp_matrix',
            'SCELPHONO' : '_sp_matrix',
            'EXTPRT'    : '_extprt',
            'CIFPRT'    : '_cifprt',
            'CIFPRTSYM' : '_cifprtsym',
            'TESTGEOM'  : '_testgeom',
            'OPTGEOM'   : 'optgeom', # Sub-blocks are called by properties defied by decorators without underscore
        }
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)
    
    def title(self, title='Generated by CRYSTALpytools'):
        self._block_bg = '{}\n'.format(title)
        
    def crystal(self, IGR=None, latt=[], atom=[], IFLAG=0, IFHR=0, IFSO=0, origin=[]):
        """
        Define 'CRYSTAL' structure
        
        Args:
            sg (int): Space group number. Parameter IGR in the manual
            latt (list): Minimal set of crystallographic cell parameters
            atom (list): Natom \* 4 list of conventional atomic number and 3D 
                fractional coordinates.
            IFLAG (int): See the manual
            IFHR (int): See the manual
            IFSO (int): See the manual
            origin (list): *IFSO > 1* See the manual
        """
        if IGR == None: # No entry. Return keyword
            self._basegeom = super(Geom, self).assign_keyword('CRYSTAL', [])
            return
        elif IGR == '': # Clean data
            self._basegeom = super(Geom, self).assign_keyword('CRYSTAL', [], '')
            return

        if IFSO <= 1:
            shape = [3, 1]
            value = [int(IFLAG), int(IFHR), IFSO, int(IGR)]
        else:
            shape = [3, 3, 1]
            value = [int(IFLAG), int(IFHR), IFSO, origin[0], origin[1], origin[2], int(IGR)]

        shape += [len(latt),]
        value += [i for i in latt]

        atominput = super(Geom, self).set_list(len(atom), atom)
        shape += atominput[0]
        value += atominput[1]

        self._basegeom = super(Geom, self).assign_keyword('CRYSTAL', shape, value)
    
    def slab(self, IGR=None, latt=[], atom=[]):
        """
        Define 'SLAB' structure
        """
        if IGR == None: # No entry. Return keyword
            self._basegeom = super(Geom, self).assign_keyword('SLAB', [])
            return
        elif IGR == '': # Clean data
            self._basegeom = super(Geom, self).assign_keyword('SLAB', [], '')
            return

        shape = [1,]
        value = [int(IGR),]

        shape += [len(latt),]
        value += [i for i in latt]

        atominput = super(Geom, self).set_list(len(atom), atom)
        shape += atominput[0]
        value += atominput[1]

        self._basegeom = super(Geom, self).assign_keyword('SLAB', shape, value)
    
    def polymer(self, IGR=None, latt=[], atom=[]):
        """
        Define 'POLYMER' structure
        """
        if IGR == None: # No entry. Return keyword
            self._basegeom = super(Geom, self).assign_keyword('POLYMER', [])
            return
        elif IGR == '': # Clean data
            self._basegeom = super(Geom, self).assign_keyword('POLYMER', [], '')
            return

        shape = [1,]
        value = [int(IGR),]

        shape += [len(latt),]
        value += [i for i in latt]
        
        atominput = super(Geom, self).set_list(len(atom), atom)
        shape += atominput[0]
        value += atominput[1]

        self._basegeom = super(Geom, self).assign_keyword('POLYMER', shape, value)
    
    def helix(self, N1=None, N2=0, latt=[], atom=[]):
        """
        Define 'HELIX' structure
        
        Args:
            N1 (int): See the manual
            N2 (int): See the manual
        """
        if N1 == None: # No entry. Return keyword
            self._basegeom = super(Geom, self).assign_keyword('HELIX', [])
            return
        elif N1 == '': # Clean data
            self._basegeom = super(Geom, self).assign_keyword('HELIX', [], '')
            return

        shape = [2,]
        value = [int(N1), int(N2),]
        
        shape += [len(latt),]
        value += [i for i in latt]

        atominput = super(Geom, self).set_list(len(atom), atom)
        shape += atominput[0]
        value += atominput[1]

        self._basegeom = super(Geom, self).assign_keyword('HELIX', shape, value)
        
    def molecule(self, IGR=None, atom=[]):
        """
        Define 'MOLECULE' structure
        """
        if IGR == None: # No entry. Return keyword
            self._basegeom = super(Geom, self).assign_keyword('MOLECULE', [])
            return
        elif IGR == '': # Clean data
            self._basegeom = super(Geom, self).assign_keyword('MOLECULE', [], '')
            return

        shape = [1,]
        value = [int(IGR),]
        
        atominput = super(Geom, self).set_list(len(atom), atom)
        shape += atominput[0]
        value += atominput[1]

        self._basegeom = super(Geom, self).assign_keyword('MOLECULE', shape, value)
    
    def external(self, key='EXTERNAL'):
        """
        Define 'EXTERNAL' structure
        """
        self._basegeom = super(Geom, self).assign_keyword(key, [])
    
    def dlvinput(self, key='DLVINPUT'):
        """
        Define 'DLVINPUT' structure
        """
        self._basegeom = super(Geom, self).assign_keyword(key, [])
        
    def supercel(self, mx=None):
        """
        Supercell by 'SUPERCEL' keyword
        
        Args:
            mx (array | list | str): ndimen \* ndimen matrix, [] or ''
        """
        shape, value = super(Geom, self).set_matrix(mx)
        self._sp_matrix = super(Geom, self).assign_keyword('SUPERCEL', shape, value)

    def supercon(self, mx=None):
        """
        Supercell by 'SUPERCON' keyword
        """
        shape, value = super(Geom, self).set_matrix(mx)
        self._sp_matrix = super(Geom, self).assign_keyword('SUPERCON', shape, value)
    
    def scelconf(self, mx=None):
        """
        Supercell by 'SCELCONF' keyword
        """
        shape, value = super(Geom, self).set_matrix(mx)
        self._sp_matrix = super(Geom, self).assign_keyword('SCELCONF', shape, value)
    
    def scelphono(self, mx=None):
        """
        Supercell by 'SCELPHONO' keyword
        """
        shape, args = super(Geom, self).set_matrix(mx)
        self._sp_matrix = super(Geom, self).assign_keyword('SCELPHONO', shape, value)
        
    def extprt(self, key='EXTPRT'):
        self._extprt = super(Geom, self).assign_keyword(key, [])
        
    def cifprt(self, key='CIFPRT'):
        self._cifprt = super(Geom, self).assign_keyword(key, [])
    
    def cifprtsym(self, key='CIFPRTSYM'):
        self._cifprtsym = super(Geom, self).assign_keyword(key, [])
        
    def testgeom(self, key='TESTGEOM'):
        self._testgeom = super(Geom, self).assign_keyword(key, [])
        
    def set_optgeom(self, obj=None):
        """
        Optgeom subblock
        
        Args:
            obj (BlockOptgeom): A block object of 'OPTGEOM' submodule.
        """
        if obj == None: # Initialize block
            self._block_optgeom = Optgeom()
        elif obj == '': # Clean data
            self._block_optgeom = ''
        else:
            self._block_optgeom = obj
    
    @property
    def optgeom(self):
        """
        Subblock object OPTGEOM
        """
        return self._block_optgeom
    
class Optgeom(BlockBASE):
    """
    OPTGEOM block object
    """
    def __init__(self):
        self._block_bg = 'OPTGEOM\n'
        self._block_ed = 'ENDOPT\n'
        self._block_data = ''
        self._block_dict = {
            'FULLOPTG' : '_opttype',
            'CELLONLY' : '_opttype',
            'INTREDUN' : '_opttype',
            'ITATOCEL' : '_opttype',
            'CVOLOPT'  : '_opttype',
            'HESSIDEN' : '_opthess',
            'HESSMOD1' : '_opthess',
            'HESSMOD2' : '_opthess',
            'HESSNUM'  : '_opthess',
            'TOLDEG'   : '_toldeg',
            'TOLDEX'   : '_toldex', 
            'TOLDEE'   : '_toldee',
            'MAXCYCLE' : '_maxcycle',
            'FRAGMENT' : '_fragment',
            'RESTART'  : '_restart',
            'FINALRUN' : '_finalrun',
            'EXTPRESS' : '_extpress',
            'ALLOWTRUSTR' : '_usetrustr',
            'NOTRUSTR'    : '_usetrustr',
            'MAXTRADIUS'  : '_maxtradius',
            'TRUSTRADIUS' : '_trustradius'
        }
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)

    def fulloptg(self, key='FULLOPTG'):
        self._opttype = super(Optgeom, self).assign_keyword(key, [])
    
    def cellonly(self, key='CELLONLY'):
        self._opttype = super(Optgeom, self).assign_keyword(key, [])
    
    def intredun(self, key='INTREDUN'):
        self._opttype = super(Optgeom, self).assign_keyword(key, [])

    def itatocel(self, key='ITATOCEL'):
        self._opttype = super(Optgeom, self).assign_keyword(key, [])
    
    def cvolopt(self, key='CVOLOPT'):
        self._opttype = super(Optgeom, self).assign_keyword(key, [])
    
    def hessiden(self, key='HESSIDEN'):
        self._opthess = super(Optgeom, self).assign_keyword(key, [])
    
    def hessmod1(self, key='HESSMOD1'):
        self._opthess = super(Optgeom, self).assign_keyword(key, [])
    
    def hessmod2(self, key='HESSMOD2'):
        self._opthess = super(Optgeom, self).assign_keyword(key, [])
    
    def hessnum(self, key='HESSNUM'):
        self._opthess = super(Optgeom, self).assign_keyword(key, [])

    def toldeg(self, TG=None):
        self._toldeg = super(Optgeom, self).assign_keyword('TOLDEG', [1,], TG)

    def toldex(self, TX=None):
        self._toldex = super(Optgeom, self).assign_keyword('TOLDEX', [1,], TX)

    def toldee(self, IG=None):
        self._toldee = super(Optgeom, self).assign_keyword('TOLDEE', [1,], IG)
    
    def maxcycle(self, MAX=None):
        self._maxcycle = super(Optgeom, self).assign_keyword('MAXCYCLE', [1,], MAX)
    
    def fragment(self, NL=None, LB=[]):
        """
        Args:
            NL (int | str): Number of atoms. See manual. Or ''
            LB (list[int]): Label of atoms. See manual
        """
        shape, value = super(Optgeom, self).set_list(NL, LB)
        self._fragment = super(Optgeom, self).assign_keyword('FRAGMENT', shape, value)

    def restart(self, key='RESTART'):
        self._restart = super(Optgeom, self).assign_keyword(key, [])
    
    def finalrun(self, ICODE=None):
        self._finalrun = super(Optgeom, self).assign_keyword('FINALRUN', [1,], ICODE)
    
    def extpress(self, pres=None):
        self._extpress = super(Optgeom, self).assign_keyword('EXTPRESS', [1,], pres)
    
    def allowtrustr(self, key='ALLOWTRUSTR'):
        self._usetrustr = super(Optgeom, self).assign_keyword(key, [])
    
    def notrustr(self, key='NOTRUSTR'):
        self._usetrustr = super(Optgeom, self).assign_keyword(key, [])
        
    def maxtradius(self, TRMAX=None):
        self._maxtradius = super(Optgeom, self).assign_keyword('MAXTRADIUS', [1,], TRMAX)
        
    def trustradius(self, TRADIUS=None):
        self._trustradius = super(Optgeom, self).assign_keyword('TRUSTRADIUS', [1,], TRADIUS)
    
class Freqcalc(BlockBASE):
    """
    FREQCALC block object
    """
    def __init__(self):
        self._block_bg = 'FREQCALC\n'
        self._block_ed = 'ENDFREQ\n'
        self._block_data = ''
        self._block_dict = {
            'NOOPTGEOM'  : 'preopt', # A tailored sub-block
            'PREOPTGEOM' : 'preopt', # A tailored sub-block
            'MODES'      : '_modes',
            'NOMODES'    : '_modes',
            'NUMDERIV'   : '_numderiv',
            'PRESSURE'   : '_pressure',
            'RESTART'    : '_restart',
            'STEPSIZE'   : '_stepsize',
            'TEMPERAT'   : '_temperat',
        }
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)
    
    def nooptgeom(self, key='NOOPTGEOM'):
        if key == 'NOOPTGEOM':
            self.preopt = Optgeom()
            self.preopt._block_bg = '{}\n'.format(key)
            self.preopt._block_ed = ''
        elif key == '':
            self.preopt.clean_block()
        else:
            raise ValueError('Wrong keyword.')
    
    def preoptgeom(self, obj=None):
        """
        Args:
            obj (Optgeom): An Optgeom block object
        """
        import warnings
        
        warnings.warn("Keyword 'PREOPTGEOM' is launched. To set geometric optimization keywords, use 'self.preopt' attribute.")
        if obj == None: # New obj
            self.preopt = Optgeom()
            self.preopt._block_bg = 'PREOPTGEOM\n'
            self.preopt._block_ed = 'END\n'
        elif obj == '':
            self.preopt.clean_block()
        else:
            self.preopt = obj
    
    def modes(self, key='MODES'):
        self._modes = super(Freqcalc, self).assign_keyword(key, [])
    
    def nomodes(self, key='NOMODES'):
        self._modes = super(Freqcalc, self).assign_keyword(key, [])
    
    def numderiv(self, N=None):
        self._numderiv = super(Freqcalc, self).assign_keyword('NUMDERIV', [1,], N)
    
    def pressure(self, NP=None, P1=None, P2=None):
        self._pressure = super(Freqcalc, self).assign_keyword('PRESSURE', [3,], [NP, P1, P2])
    
    def restart(self, key='RESTART'):
        self._restart = super(Freqcalc, self).assign_keyword('RESTART', [])
    
    def stepsize(self, STEP=None):
        self._stepsize = super(Freqcalc, self).assign_keyword('NUMDERIV', [1,], STEP)
    
    def temperat(self, NT=None, T1=None, T2=None):
        self._temperat = super(Freqcalc, self).assign_keyword('TEMPERAT', [3,], [NT, T1, T2])
    # def dispersion
    
class BasisSet(BlockBASE):
    """
    Basis Set block object
    """
    def __init__(self):
        self._block_bg = ''
        self._block_ed = '99 0\nENDBS\n'
        self._block_data = ''
        self._block_dict = {
            'BASISSET' : '_basisset',
            'GHOSTS'   : '_ghosts',
        }
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)
    
    def basisset(self, NAME=None):
        self._basisset = super(BasisSet, self).assign_keyword('BASISSET', [1,], NAME)
        # Otherwise _block_bg and _block_ed = '', this block would be recoginzed as an empty block
        if NAME == '':
            self._block_ed = '99 0\nENDBS\n'
        else:
            self._block_ed = '\s\b'
    
    def bs_from_string(self, string):
        """
        Args:
            string (str): A line of string. Use '\n' to break lines. The ending
            lines '99 0' and 'END' are not needed.
        """
        import re, warnings

        if 'BASISSET' in self._basisset:
            warnings.warn("The 'BASISSET' keyword is in use. It will be cleaned.")
            self._block_ed = '99 0\nENDBS\n'

        value = string.split('\n')
        for v in value:
            if re.match(r'^99\s+0', v) or re.match(r'^END.*', v):
                value.remove(v)
        shape = [1 for v in value]
        self._basisset = super(BasisSet, self).assign_keyword('\s\b', shape, value) # Avoid empty keyword

    def bs_from_file(self, file):
        """
        Args:
            file (file): A formatted text file with basis set definitions. The 
            ending lines '99 0' and 'END' are not needed.
        """
        import warnings

        if 'BASISSET' in self._basisset:
            warnings.warn("The 'BASISSET' keyword is in use. It will be cleaned.")
            self._block_ed = '99 0\nENDBS\n'

        bs = open(file, 'r')
        value = bs.readlines()
        for v in value:
            if re.match(r'^99\s+0', v) or re.match(r'^END.*', v):
                value.remove(v)
        shape = [1 for v in value]
        self._basisset = super(BasisSet, self).assign_keyword('\s\b', shape, value) # Avoid empty keyword
    
    def ghosts(self, NA=None, LA=[]):
        shape, value = super(BasisSet, self).set_list(NA, LA)
        self._ghosts = super(BasisSet, self).assign_keyword('GHOSTS', shape, value)

class SCF(BlockBASE):
    """
    SCF block object
    """
    def __init__(self):
        self._block_bg = ''
        self._block_ed = 'ENDSCF\n'
        self._block_data = ''
        self._block_dict = {
            'DFT'      : 'dft', # DFT sub-block
            'BIPOSIZE' : '_biposize',
            'EXCHSIZE' : '_exchsize',
            'FIXINDEX' : 'fixindex', # A tailored sub-block
            'TOLDEE'   : '_toldee',
            'GUESSP'   : '_guessp',
            'ATOMSPIN' : '_atomspin',
            'TOLINTEG' : '_tolinteg',
            'LDREMO'   : '_ldremo',
            'MAXCYCLE' : '_maxcycle',
            'FMIXING'  : '_fmixing',
            'SHRINK'   : '_shrink',
        }
        key = list(self._block_dict.keys())
        attr = list(self._block_dict.values())
        self._block_key = sorted(set(key), key=key.index)
        self._block_attr = sorted(set(attr), key=attr.index)
    
    def biposize(self, ISIZE=None):
        self._biposize = super(SCF, self).assign_keyword('BIPOSIZE', [1,], ISIZE)
    
    def exchsize(self, ISIZE=None):
        self._exchsize = super(SCF, self).assign_keyword('EXCHSIZE', [1,], ISIZE)
    
    # def fixindex()
    
    def toldee(self, ITOL=None):
        self._toldee = super(SCF, self).assign_keyword('TOLDEE', [1,], ITOL)
    
    def guessp(self, key='GUESSP'):
        self._guessp = super(SCF, self).assign_keyword(key, [])
    
    def atomspin(self, NA=None, LA=[]):
        shape, value = super(SCF, self).set_list(NA, LA)
        self._atomspin = super(SCF, self).assign_keyword('ATOMSPIN', shape, value)
    
    def tolinteg(self, ITOL1=None, ITOL2=None, ITOL3=None, ITOL4=None, ITOL5=None):
        self._tolinteg = super(SCF, self).assign_keyword(
            'TOLINTEG', [5,], [ITOL1, ITOL2, ITOL3, ITOL4, ITOL5])
    
    def ldremo(self, value):
        self._ldremo = super(SCF, self).assign_keyword('LDREMO', [1,], value)
    
    def maxcycle(self, MAX=None):
        self._maxcycle = super(SCF, self).assign_keyword('MAXCYCLE', [1,], MAX)
    
    def fmixing(self, IPMIX=None):
        self._maxcycle = super(SCF, self).assign_keyword('FMIXING', [1,], IPMIX)
    
    def shrink(self, IS=None, ISP=None, IS1=None, IS2=None, IS3=None):
        if IS1 == None:
            self._shrink = super(SCF, self).assign_keyword(
                'SHRINK', [2,], [IS, ISP])
        else:
            self._shrink = super(SCF, self).assign_keyword(
                'SHRINK', [2, 3], [IS, ISP, IS1, IS2, IS3])


In [166]:
geometry = Geom()
geometry.title('Form2paracetamol optimisation Step m4 - PBEH3CBJ/PBEh-3C')

In [169]:
geometry.crystal(61, [7.0776, 16.9653, 11.6318], [
    [1, 0.07049295, 0.23113798, 0.43012291],
    [1, 0.08359712, 0.14827962, 0.03642604],
    [1, 0.12306322, 0.51098435, 0.84606520],
    [1, 0.13165550, 0.73503486, 0.80631766],
    [1, 0.14381188, 0.03823873, 0.87012518]])
geometry.supercel([[2, 0, 0],
                   [0, 2, 0],
                   [0, 0, 2]])
geometry.set_optgeom()
geometry.optgeom.toldeg(0.0003)
geometry.optgeom.cvolopt()

In [170]:
geometry.data

Form2paracetamol optimisation Step m4 - PBEH3CBJ/PBEh-3C
CRYSTAL
0 0 0
61
7.0776 16.9653 11.6318
5
1 0.07049295 0.23113798 0.43012291
1 0.08359712 0.14827962 0.03642604
1 0.12306322 0.51098435 0.8460652
1 0.1316555 0.73503486 0.80631766
1 0.14381188 0.03823873 0.87012518
SUPERCEL
2 0 0
0 2 0
0 0 2
OPTGEOM
CVOLOPT
TOLDEG
0.0003
ENDOPT
ENDGEOM

